In [1]:
import pandas as pd
from sklearn.metrics import matthews_corrcoef,precision_score,recall_score,f1_score

In [56]:
d=pd.read_csv('babe_test_annomatic.csv')

In [57]:
dd=d[['text','label','zephyr_label','openchat_label','llama_13b_label']]
dd=dd[dd['zephyr_label']!="?"]

dd['zephyr_label'] = dd['zephyr_label'].astype(int)
dd['openchat_label'] = dd['openchat_label'].astype(int)
dd['llama_13b_label'] = dd['llama_13b_label'].astype(int)

In [58]:
dd['majority']  =dd[['zephyr_label','openchat_label','llama_13b_label']].mode(axis=1)

In [59]:
matthews_corrcoef(y_true=dd['label'],y_pred=dd['majority'])

0.6390586829640236

In [18]:
# synth preds

In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mediabiasgroup/roberta-anno-lexical-ft")
model = AutoModelForSequenceClassification.from_pretrained("mediabiasgroup/roberta-anno-lexical-ft")

/home/tomas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
device = (
    torch.device("cuda:0")
    if torch.cuda.is_available()
    else torch.device("cpu")
)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [22]:
from abc import ABC, abstractmethod
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
)
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
import pandas as pd

In [27]:
def make_predictions(data=None, target_col: str = "text"):
        """
        Generates predictions for the test data using the trained model.

        Returns:
            predictions (list): List of predicted labels for the test data.
        """

        tok = tokenizer(
            list(data[target_col]),
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors="pt",
        )
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        testing_dataloader = DataLoader(
            Dataset.from_dict(tok), batch_size=16, collate_fn=data_collator
        )

        predictions = []
        for batch in tqdm(testing_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).tolist())

        return predictions

In [28]:
preds = make_predictions(dd)

100%|██████████| 63/63 [02:33<00:00,  2.43s/it]


In [29]:
dd['synth'] = preds

/tmp/ipykernel_13613/2883807505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['synth'] = preds


In [34]:
matthews_corrcoef(y_true=dd['label'],y_pred=dd['majority'])

0.6390586829640236